# KVLCC2 Ikeda method

# Purpose
How good is original Ikeda method for this ship?

# Methodology
Run PyScoresII and calculate Ikeda

# WIP - improvements
(WORK IN PROGRESS)
Use this section only if the notebook is not final.

Notable TODOs:
* todo 1
* todo 2
* todo 3

## Results
Describe and comment the most important results.

# Suggested next steps
State suggested next steps, based on results obtained in this notebook.

# Setup

In [ ]:
# %load imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import OrderedDict
#plt.style.use('paper')

#import data
import copy
from mdldb.run import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

from sklearn.metrics import r2_score
from src.data import database
from mdldb import tables


In [ ]:
import pyscores2
import pyscores2.runScores2
import pyscores2.xml_hydrostatics
from pyscores2.output import OutputFile
from rolldecayestimators.ikeda import Ikeda, IkedaR

from rolldecayestimators.simplified_ikeda_class import SimplifiedIkeda, SimplifiedIkedaABS
from rolldecayestimators.simplified_ikeda import limits_kawahara
from pyscores2.runScores2 import Calculation
import shutil

In [ ]:
db = database.get_db()

In [ ]:
sql = """
SELECT * from run
WHERE model_number='M5057-01-A' and test_type='roll decay' and project_number=40178362;
"""
df_rolldecays = pd.read_sql(sql=sql, con=db.engine, index_col='id')

df_rolldecays['ship_speed'].fillna(0, inplace=True)


In [ ]:
df_rolldecays=df_rolldecays.loc[[21337,21338]].copy()

In [ ]:
row = df_rolldecays.iloc[0]
run = db.session.query(Run).get(int(row.name))
run = database.load_run(run, save_as_example=False, prefer_hdf5=True)
  

In [ ]:
print(run.project.project_path)

## Run ScoresII

In [ ]:
xml_parser = pyscores2.xml_hydrostatics.Parser(fileName='../data/external/KVLCC2m_kbk_final_ScoresData.xml')
indata = xml_parser.convertToScores2Indata(conditionName='Design')
indata.runOptions["IJ"].set_value(1)
indata.runOptions["IK"].set_value(2)

In [ ]:
indata.kxx = run.loading_condition.KXX*0.78  # To get correct natural frequency
indata.kyy = run.loading_condition.KZZ
indata.speedMax=15.5
indata.speedIncrement=15
indata.waveFrequenciesMax = 1.0
indata.waveFrequenciesMin = 0.3
indata.waveFrequenciesIncrement = 0.015
#indata.zcg = run.loading_condition.kg

In [ ]:
indata.save('../models/KVLCC2_0speed.IN')

In [ ]:
save_dir_name = 'scores'
if not os.path.exists(save_dir_name):
    os.mkdir(save_dir_name)
    
calculation = Calculation(outDataDirectory='scores')

In [ ]:
calculation.run(indata=indata)

## Load ScoresII results

In [ ]:
output_file = OutputFile(filePath=calculation.outDataPath)

In [ ]:
shutil.copyfile(calculation.outDataPath,'../data/interim/KVLCC2_0speed.out')

In [ ]:
output_file.results

In [ ]:
df = output_file.get_result()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df[r'lambda/lpp'] = df['waveLengths']/run.ship.lpp 


fig,ax=plt.subplots()
for index, group in df.groupby(by=['speed','wave direction']):
    group.plot(x=r'lambda/lpp', y='heaveAmplitude', style='o-', label=index, ax=ax)
    
ax.grid(True)
ax.legend();
ax.set_ylabel('Heave');

In [ ]:
RAO_15_0 = df.groupby(by=['speed','wave direction']).get_group((15,180))

In [ ]:
fig,ax=plt.subplots()
RAO_15_0.plot(x=r'lambda/lpp', y='heaveAmplitude', style='o-', ax=ax)
ax.set_xlim((0,1.8))

fig,ax=plt.subplots()
RAO_15_0.plot(x=r'lambda/lpp', y='pitchAmplitude', style='o-', ax=ax)
#ax.set_xlim((0,1.8))

In [ ]:
df_roll_damping = output_file.get_roll_damping()
df_roll_damping

## Run Ikeda

In [ ]:
w = 2.462149630662348

scale_factor=run.model.scale_factor
V = row.ship_speed*1.852/3.6/np.sqrt(scale_factor)

if not run.ship.BKL:
    BKL=0
else:
    BKL=run.ship.BKL/scale_factor

if not run.ship.BKB:
    BKB = 0
else:
    BKB=run.ship.BKB/scale_factor

#fi_as = np.deg2rad([1,10])
fi_as = np.deg2rad(10)

ikeda = Ikeda.load_scoresII(V=V, w=w, fi_a=fi_as, indata=indata, output_file=output_file, 
                            scale_factor=scale_factor, BKL=BKL, BKB=BKB)

R = 0.05*run.ship.beam/scale_factor  # Just guessing...
ikeda.R = R

In [ ]:
def calculate_ikeda(ikeda):

    output = pd.DataFrame()
    output['B_44_hat']   = ikeda.calculate_B44()
    output['B_W0_hat']   = float(ikeda.calculate_B_W0())
    output['B_W_hat']    = float(ikeda.calculate_B_W())
    output['B_F_hat']    = ikeda.calculate_B_F()
    output['B_E_hat']    = ikeda.calculate_B_E()
    output['B_BK_hat']   = ikeda.calculate_B_BK()
    output['B_L_hat']    = float(ikeda.calculate_B_L())
    output['Bw_div_Bw0'] = float(ikeda.calculate_Bw_div_Bw0())
    return output

In [ ]:
result_datas = calculate_ikeda(ikeda)  # DataFrame with two roll amplitudes

In [ ]:
result_datas

## Simplified Ikeda also...

In [ ]:
lpp = run.ship.lpp/scale_factor
beam = run.ship.beam/scale_factor
kg = run.loading_condition.kg/scale_factor
volume = run.loading_condition.Volume/(scale_factor**3)
draught = (run.loading_condition.TA + run.loading_condition.TF)/2/scale_factor
A0 = run.loading_condition.A0

if not run.ship.BKL:
    BKL=0
else:
    BKL = run.ship.BKL

if not run.ship.BKB:
    BKB = 0
else:
    BKB = run.ship.BKB

si = SimplifiedIkeda(V=V, w=w, fi_a=fi_as, beam=beam, lpp=lpp, kg = kg, volume=volume, draught=draught, A0=A0, BKL=BKL, BKB=BKB)

In [ ]:
def calculate_SI(si):
    
    output = pd.DataFrame()
    output['B_44_hat'] = si.calculate_B44()
    output['B_W0_hat'] =si.calculate_B_W0()
    output['B_W_hat'] =si.calculate_B_W()
    output['B_F_hat'] =si.calculate_B_F()
    output['B_E_hat'] =si.calculate_B_E()
    output['B_BK_hat'] =si.calculate_B_BK()
    output['B_L_hat'] =si.calculate_B_L()
    output['Bw_div_Bw0'] =si.calculate_Bw_div_Bw0()
    
    return output

In [ ]:
result_datas_SI = calculate_SI(si=si)

In [ ]:
result_datas_SI

In [ ]:
df_results = pd.DataFrame(columns=result_datas.columns)
df_results.loc['ikeda']=result_datas.iloc[0]
df_results.loc['SI']=result_datas_SI.iloc[0]


<a id='eddy'></a>

In [ ]:
interesting = ['B_W_hat','B_F_hat','B_E_hat']
df_results[interesting].plot(kind='bar',stacked=True)

In [ ]:
limits_kawahara

In [ ]:
df_limits = pd.DataFrame(data = limits_kawahara, index = ['min','max']).transpose()
df_limits

In [ ]:
g=9.81
omega_hat = lambdas.omega_hat(beam=beam, g=g, omega0=w)

In [ ]:
Cb = volume/(lpp*beam*draught)
OG = draught-kg
ship_limits = {
 'CB': Cb,
 'B/d': beam/draught,
 'OG/d': OG/draught,
 'CMID': A0,
 'bBk/B': BKB/beam,
 'lBk/LPP': BKL/beam,
 'OMEGA_hat': omega_hat}
ship_limits = pd.Series(ship_limits,name='ship')

In [ ]:
df_limits['ship'] = ship_limits

In [ ]:
df_limits_clean = df_limits.copy()
if df_limits.loc['bBk/B','ship']==0:
    df_limits_clean.drop('bBk/B', inplace=True)
    
if df_limits.loc['lBk/LPP','ship']==0:
    df_limits_clean.drop('lBk/LPP', inplace=True)

In [ ]:
fig,ax=plt.subplots()
ax.errorbar(df_limits_clean.index,df_limits_clean['ship'],yerr=[df_limits_clean['ship']-df_limits_clean['min'],df_limits_clean['max']-df_limits_clean['ship']], 
            fmt='ok', lw=1, ecolor='gray', capsize=20)
ax.set_title('Ship vs. SI limits')

In [ ]:
df_ = df_limits_clean.sub(df_limits['min'],axis=0)
df_limits_normalized = df_.div(df_['max'], axis=0)

<a id='limits_kawahara'></a>

In [ ]:
fig,ax=plt.subplots()
ax.errorbar(df_limits_normalized.index,df_limits_normalized['ship'],yerr=[df_limits_normalized['ship']-df_limits_normalized['min'],
                                                                         df_limits_normalized['max']-df_limits_normalized['ship']], fmt='ok', lw=1, ecolor='gray', capsize=20)
ax.set_title('Ship vs. SI limits')
ax.set_ylabel('Norlimized limit')